In [1]:
import sys
sys.path.append('/home/huangbo/Desktop/objectdetection/') #Give local path to objectdetection repository
import os.path
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import model_from_json
from keras.backend import set_image_dim_ordering
set_image_dim_ordering('tf')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


In [2]:
model_dir = "/home/huangbo/objectdetection/objectdetection/huangbo_ws/models/06.18_resnet_256/"
model_definition = model_dir + "model.json"
model_weights = model_dir + "model.hdf5"

mean = np.array([114.96066323, 116.50405346, 102.74354111])

In [3]:
from objectdetection.utils.ml.multi_gpu import load_and_transfer
model = load_and_transfer(model_file=model_definition, weights_file=model_weights)

In [3]:
import yaml
yaml_file = "/home/huangbo/HuangBo_Projects/data/nordhorn/dataset_500/nordhorn_home.yml"
spec = yaml.load(open(yaml_file, "rb").read())
file_names = spec["validation"]["images"]
masks_names = spec["validation"]["labels"]

In [15]:
from objectdetection.metrics import semseg_metric

In [22]:
total_pred = np.int64(0)
total_improved = np.int64(0)
total_pixels = np.int64(0)

ious_cnn = []
for i, _ in enumerate(file_names):

    image_path = file_names[i]
    img = cv2.imread(image_path, cv2.IMREAD_COLOR | cv2.IMREAD_ANYDEPTH)
    
    img = img[0:256, 0:256, :]

    img = img - mean
    img = img / 255.
    img = np.expand_dims(img, axis=0)
    
    masks_path = masks_names[i]
    gt = cv2.imread(masks_path, -1)
    gt = gt[0:256, 0:256]
    cnn_pred = model.predict(img)
    cnn_pred = cnn_pred[0].argmax(-1).reshape(256, 256)
    
#     plt.subplot(1,3,1)
#     plt.imshow(img[0, :, :, ::-1])
#     plt.subplot(1,3,2)
#     plt.imshow(gt)
#     plt.subplot(1,3,3)
#     plt.imshow(cnn_pred)
    
    
    iou_cnn = semseg_metric.batch_iou(cnn_pred, gt, 1, 1)
    ious_cnn.extend(iou_cnn)
   
    total_pred += sum(sum(cnn_pred == gt))
    total_pixels += 256*256
    

ious = np.asarray(ious_cnn)
intersection, union = np.split(np.sum(ious, axis=0), 2, axis=0)
iou_cnn_score = np.true_divide(intersection, union)


print iou_cnn_score
print np.float128(total_pred) / np.float128(total_pixels)

[[ 0.64770141]]
0.864098270244


In [34]:
import pandas as pd
from sklearn.metrics import confusion_matrix
total_pred = np.int64(0)
total_improved = np.int64(0)
total_pixels = np.int64(0)

ious_cnn = []
cm = np.zeros(shape=(11,11))
for i, _ in enumerate(file_names):

    image_path = file_names[i]
    img = cv2.imread(image_path, cv2.IMREAD_COLOR | cv2.IMREAD_ANYDEPTH)
    
    img = img[0:256, 0:256, :]

    img = img - mean
    img = img / 255.
    img = np.expand_dims(img, axis=0)
    
    masks_path = masks_names[i]
    gt = cv2.imread(masks_path, -1)
    gt = gt[0:256, 0:256]
    cnn_pred = model.predict(img)

    gt = gt.flatten()
    cnn_pred = cnn_pred[0].argmax(-1).reshape(256*256)
    cm_new = confusion_matrix(y_true=gt, y_pred=cnn_pred, labels=range(11))
    
    cm = cm + cm_new

print 'The confusion_matrix:'
print cm
# total accuracy
tp = np.trace(cm)
tp = float(tp)
accuracy = tp / np.sum(cm)
print 'the total acc is:', accuracy

# recall and precision

recall = np.zeros(11, dtype=float)
precision = np.zeros(11, dtype=float)
f1 = np.zeros(11, dtype=float)
nb_samples = np.zeros(11)

for i in range(11):
    fp = np.sum(cm[:, i]) - cm[i, i]
    fn = np.sum(cm[i, :]) - cm[i, i]

    if cm[i, i] + fp == 0:
        recall[i] = 0
    else:
        recall[i] = cm[i, i] / (cm[i, i] + fp)

    if cm[i, i] + fn == 0:
        precision[i] = 0
    else:
        precision[i] = cm[i, i] / (cm[i, i] + fn)

for i in range(11):
    nb_samples[i] = np.sum(cm[:, i])
    if precision[i] + recall[i] == 0:
        f1[i] = 0
    else:
        f1[i] = 2 * precision[i] * recall[i] / (precision[i] + recall[i])

where_are_NaNs = np.isnan(recall)
recall[where_are_NaNs] = 0

where_are_NaNs = np.isnan(precision)
precision[where_are_NaNs] = 0

dic_report = {"recall": recall,
              "precision": precision,
              "f1_score": f1,
              "number_samples": nb_samples
              }

labels_index = ['background', 'building', 'asphalt/concrete', 'railway',
                  'cars', 'flat vegetation', 'bushes (medium vegetation)',
                  'trees (high vegetation)', 'water', 'fallow land', 'sand / rock']


print pd.DataFrame(dic_report, index=labels_index)

The confusion_matrix:
[[  3.98000000e+02   2.99500000e+03   7.26700000e+03   6.47000000e+02
    0.00000000e+00   8.29980000e+04   5.50000000e+01   1.74710000e+04
    8.57000000e+02   2.08980000e+04   2.06000000e+02]
 [  4.08000000e+02   7.09486000e+05   7.29760000e+04   1.00000000e+00
    0.00000000e+00   1.20073000e+05   5.57000000e+02   1.34020000e+04
    1.56620000e+04   3.86800000e+03   8.28100000e+03]
 [  4.21100000e+03   1.52747000e+05   1.41167500e+06   0.00000000e+00
    1.00000000e+00   1.75363000e+05   2.34100000e+03   7.40300000e+04
    4.30000000e+02   1.48494000e+05   1.53670000e+04]
 [  0.00000000e+00   1.54000000e+02   0.00000000e+00   4.55050000e+04
    0.00000000e+00   5.97200000e+03   0.00000000e+00   0.00000000e+00
    7.90000000e+01   0.00000000e+00   4.16000000e+02]
 [  0.00000000e+00   7.44500000e+03   2.14700000e+03   0.00000000e+00
    1.04000000e+02   3.80000000e+01   0.00000000e+00   1.40000000e+01
    0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.82

In [57]:
image_path = file_names[0]
img = cv2.imread(image_path, cv2.IMREAD_COLOR | cv2.IMREAD_ANYDEPTH)
img1 = img[0:250,0:250,:]
img2 = img[250:500,0:250,:]
img3 = img[0:250,250:500,:]
img4 = img[250:500,250:500,:]

img1 = cv2.resize(img1, (252,252), 3)
img2 = cv2.resize(img2, (252,252), 3)
img3 = cv2.resize(img3, (252,252), 3)
img4 = cv2.resize(img4, (252,252), 3)



(252, 252, 3)
(252, 252, 3)
(252, 252, 3)
(252, 252, 3)


In [ ]:
 
img1 = img[0:250,0:250,:]
img2 = img[250:500,0:250,:]
img3 = img[0:250,250:500,:]
img4 = img[250:500,250:500,:]

img1 = cv2.resize(img1, (252,252), 3)
img2 = cv2.resize(img2, (252,252), 3)
img3 = cv2.resize(img3, (252,252), 3)
img4 = cv2.resize(img4, (252,252), 3)